## ResNet50 
ResNet-50 is a convolutional neural network that is 50 layers deep. One can load a pretrained version of the neural network trained on more than a million images from the ImageNet database.

In [4]:
# Import required packages
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the seed for reproducibility
tf.random.set_seed(42)

In [5]:
# Set the paths to train and valid folders
train_path = r'C:\Users\KIIT\PlantDatasetimages\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\train'
valid_path = r'C:\Users\KIIT\PlantDatasetimages\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\valid'

# Get list of all subfolders for each subset
train_dir = os.listdir(train_path)
valid_dir = os.listdir(valid_path)

print(len(train_dir), len(valid_dir))

38 38


In [6]:
# Set the image size and batch size
image_size = (224, 224)
batch_size = 32

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Create ImageDataGenerator instances with data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [8]:
# Load and preprocess the training dataset
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Load and preprocess the validation dataset
valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 56251 images belonging to 38 classes.
Found 3503 images belonging to 38 classes.


In [9]:
# Check the number of classes
num_classes = len(train_generator.class_indices)

In [10]:
from tensorflow.keras.applications import ResNet50
# Load the pre-trained ResNet50 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze the base model's layers
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_from = 100

# Freeze layers before fine-tune_from
for layer in base_model.layers[:fine_tune_from]:
    layer.trainable = False

In [11]:
# Create the new model architecture by adding the top layers
model_resnet50 = tf.keras.Sequential([
base_model,
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(128, activation='relu'),
tf.keras.layers.Dense(num_classes, activation='softmax')
])
# Compile the model
model_resnet50.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

# Define a learning rate schedule
def lr_schedule(epoch):
    initial_lr = 0.001
    if epoch < 10:
        return initial_lr
    else:
        return initial_lr * tf.math.exp(0.1 * (10 - epoch))

# Create a learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

# Train the model with the learning rate scheduler
history_resnet50 = model_resnet50.fit(
    train_generator,
    epochs=20,
    validation_data=valid_generator,
    callbacks=[lr_scheduler]
)

Epoch 1/20
 440/1758 [======>.......................] - ETA: 1:21:13 - loss: 3.8485 - accuracy: 0.0254

In [ ]:
# Save the model
save_dir = r'C:\Users\KIIT\PlantDatasetimages'
save_path_resnet50 = os.path.join(save_dir, 'model_resnet50.h5')
model_resnet50.save(save_path_resnet50)
